<a href="https://colab.research.google.com/github/lucaliverani76/Chess_with_DQN/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


%cd "/content/drive/MyDrive/Colab Notebooks/Chess"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Chess


In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/Chess/Chessx.ipynb"



Running on CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.9 MB/s eta 0:00:00
cpu


In [ ]:
import random
import numpy as np
import collections
import pickle
import os


import glob
import re

import matplotlib.pyplot as plt



# sole purpose of this function is to upload data from previous Colab session
def findoldcalcs(extn):
    txtfiles = []

    for file in glob.glob(extn):
        txtfiles.append(file)

    x=[]
    for txt in txtfiles:
        x=x+re.findall(r'\d+', txt)
    #print(x)
    x=[int(y) for y in x]
    if len(x)>0:
        result=[max(x)]
    else:
        result=[]

    if len(result)>0:
        for n,y in enumerate(x):
            #print(n,y)
            if y!=max(x):
              #print(txtfiles[n])
              os.remove(txtfiles[n])
        return int(result[0])
    else:
        return int(0)

#this function looks if previous sessions of colab crushed and restart the system from there
def Dogenericstuff(Q_action_value):
    Savedweights_=findoldcalcs("*.pth")
    episode=np.floor(Savedweights_/1000)
    minibatch_sample=Savedweights_-episode*1000

    if episode!=0 or minibatch_sample!=0:
        # loading weights
        Q_action_value.wpath=weightfile+str(int(episode*1000+minibatch_sample))+'.pth'
        #print(Q_target_value.wpath)
        Q_action_value.loadWeights()
        copyweightsfromto_Q_action_value_to_Q_target_value()

    return episode, minibatch_sample


def Savingweights():
    # saving weights
    Q_action_value.wpath=weightfile+str(int(episode*1000+minibatch_sample))+'.pth'
    #print(Q_action_value.wpath)
    Q_action_value.SaveWeights()


def Defineorload_reply_memory(N):
    n=findoldcalcs("*.pickle")  # sole purpose of taking over where we left
    print(n)

    if n>0:
      with open('Replaymemory'+str(n)+'.pickle', 'rb') as handle:
          D = pickle.load(handle)
    else:
        D=collections.deque(maxlen=N)
    return n,D

def Save_reply_memory():
    if (n)%1000==0:
        print("Replay memory reached {} units".format(n))
        with open('Replaymemory'+str(n)+'.pickle', 'wb') as handle:
            pickle.dump(D, handle, protocol=pickle.HIGHEST_PROTOCOL)

def ResetReplymemory():
    txtfiles = []
    for file in glob.glob("*.pickle"):
        txtfiles.append(file)
    for f in txtfiles:
        os.remove(f)

def ResetWeight_History():
    txtfiles = []
    for file in glob.glob("*.pth"):
        txtfiles.append(file)
    for f in txtfiles:
        os.remove(f)

def Trytogetoldweights():

    try:
      Q_action_value.wpath=weightfile +'.pthx'
      Q_action_value.loadWeights()
      copyweightsfromto_Q_action_value_to_Q_target_value()
      print("loaded old dictionary")

    except:
      pass

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# =============================================================================
# Algorithm DQN
# =============================================================================

T=30 #number of action steps, or games moves


epsilon=0.5 #greedy policy
decay = 0.95
min_epsilon = 0.2



 # initialize replay memory with size N
N=10000

lambda_=0.3 #discount factor

N_minibatch_sampling=1000

n_tests_on_batch=int(N/N_minibatch_sampling*1.5)

N_episodes=5

weightfile='model_weights_tanh_moreneurons_Hdiscountf'

torch.set_printoptions(precision=10)


initializeAgents()  #simply we initialize the weights of value functions


Trytogetoldweights()




cpu
loaded old dictionary


In [ ]:

episode, minibatch_sample= Dogenericstuff(Q_action_value)




while episode < N_episodes:

    print("episode {0}".format(episode))

    # I fill the Replay memory
    n,D=Defineorload_reply_memory(N)


    while n<N:

        #sequences can start from always the initial
        #state or in a random state, depending how we build the fuction initialize_sequence()

        S=initialize_sequences()
        white=True



        for j in range(T):

            possible_actions=PossibleActions(S)

            if (possible_actions==[] or possible_actions==None):
                break

            if random.uniform(0, 1)<epsilon:
                #especially at the beginning where we have no experince we tend to choose random actions
                action=possible_actions[random.randrange(len(possible_actions))]
            else:
                #we evaluate the best possible action given the current state
                action_values =eval_Q_action_value(S,possible_actions)

                if white:
                    i=np.argmax(action_values)
                else:
                    i=np.argmin(action_values)

                action=possible_actions[i]

            S_next=Implement_action(S,action)

            reward=Reward(S)

            if white:
                D.append([S,action,reward,S_next])
                n=n+1
                Save_reply_memory()



            S=S_next


            white=not white

            if S.loc[0,"isover"]:
                break;



    print("End replay memory")
        # current_replay_memory_ratio=int(np.ceil(len(D)/N_minibatch_sampling))





    while minibatch_sample < n_tests_on_batch:

        print("Test io  minibatch {0}".format(minibatch_sample))

        S_s,action_s,reward_s,S_next_s =extract_minibatch(D, N_minibatch_sampling)  #extract data from batch

        y = (reward_s+ lambda_* Max_Q_target_value(S_next_s)).astype(np.float32)

        # trying to optimize the fit between Q_action_value and y
        R=OptimizeQ_action_value(S_s,y)

        Savingweights()

        minibatch_sample=minibatch_sample+1


    minibatch_sample=0

    #at the end of every training we align the target and action value function
    # hoping that these asyntotic optimization will converge on something meaningful
    copyweightsfromto_Q_action_value_to_Q_target_value()

    # I start reducing the random moves and I follow more
    #and more the Q function advises
    epsilon = max(min_epsilon, epsilon*decay)

    n=0
    ResetReplymemory()


    episode=episode+1





#And finally
ResetWeight_History()
Q_action_value.wpath=weightfile+'.pthx'
Q_action_value.SaveWeights()





episode 0.0
10000
End replay memory
Test io  minibatch 0.0
Loss: 7.410744728986174e-08
Test io  minibatch 1.0
Loss: 6.125810614321381e-08
Test io  minibatch 2.0
Loss: 5.202661412795351e-08
Test io  minibatch 3.0


# New Section

In [ ]:
R1=R.clone().detach().cpu().numpy()
output_np = R1
# Plot input and output
plt.figure(figsize=(10, 5))


plt.plot(output_np, label='NN')

plt.plot(y+0.00001, label='Rewards')
plt.title('Input Data')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')

